# METAS UncLib Python Tutorial

Michael Wollensack METAS - 02.05.2019

## Linear uncertainty propagation

Import [METAS UncLib](https://www.metas.ch/unclib) and use the linear uncertainty propagation.

In [1]:
from metas_unclib import *
use_linprop()

### Create uncertain numbers

In [2]:
x1 = ufloat(1.0, 0.1, desc='x1')
print(x1)

1.0 ± 0.1


In [3]:
x2 = ufloat(2.0, 0.2, desc='x2')
print(x2)

2.0 ± 0.2


### Create uncertain arrays

In [4]:
v1 = ufloatarray([1.0, 2.0], np.diag([0.1**2, 0.2**2]))
print(v1)

[1.0 ± 0.1 2.0 ± 0.2]


or

In [5]:
v2 = np.array([x1, x2])
print(v2)

[1.0 ± 0.1 2.0 ± 0.2]


### Create uncertain matrices

The order in the covariance matrix is based on a column-wise vectorization of the matrix.

In [6]:
m1 = ufloatarray([[1.0, 2.0], [3.0, 4.0]], np.diag([0.1**2, 0.3**2, 0.2**2, 0.4**2]))
print(m1)

[[1.0 ± 0.1 2.0 ± 0.2]
 [3.0 ± 0.3 4.0 ± 0.4]]


In [7]:
m2 = np.array([[x1, x2], [3.*x1, 2.*x2]])
print(m2)

[[1.0 ± 0.1 2.0 ± 0.2]
 [3.0 ± 0.30000000000000004 4.0 ± 0.4]]


### Careful: The matrices look the same but they are different

m1 and m2 have the same values but different dependencies.
The elements of m1 are uncorrelated base inputs whereas some of the elements of m2 were constructed from base inputs x1 and x2 and are thus correlated. The difference is obvious if one looks at the uncertainties of the inverted matrices.

In [8]:
m3 = ulinalg.inv(m1)
print(m3)

[[-1.9999999999999998 ± 1.1135528725660042 1.0 ± 0.4582575694955839]
 [1.4999999999999998 ± 0.6873863542433759
  -0.49999999999999994 ± 0.278388218141501]]


In [9]:
m4 = ulinalg.inv(m2)
print(m4)

[[-1.9999999999999998 ± 0.2 1.0 ± 0.10000000000000003]
 [1.4999999999999998 ± 0.15 -0.49999999999999994 ± 0.04999999999999999]]


### Do some calculations with uncertain numbers

In [10]:
x3 = x1 + x2
print(x3)

3.0 ± 0.223606797749979


In [11]:
x4 = x1 * x2
print(x4)

2.0 ± 0.28284271247461906


In [12]:
x5 = umath.sqrt(x3 + x4)
print(x5)

2.23606797749979 ± 0.11180339887498948


In [13]:
x6 = x3 / x4
print(x6)

1.5 ± 0.1118033988749895


In [14]:
x7 = x5 * umath.sin(x6)
print(x7)

2.2304665972619078 ± 0.09603222054858558


### Return expectation value of result

In [15]:
print(get_value(x7))

2.2304665972619078


for linear uncertainty propagation the same as function value.

In [16]:
print(get_fcn_value(x7))

2.2304665972619078


### Calculate uncertainty of result

In [17]:
print(get_stdunc(x7))

0.09603222054858558


### Calculate expanded uncertainty of result

returns lower and upper limit.

In [18]:
print(get_coverage_interval(x7, 0.95))

[[2.0422469  2.41868629]]


### Calculate covariance or correlation of result

In [19]:
print(get_covariance([x6, x7]))

[[ 0.0125     -0.00917517]
 [-0.00917517  0.00922219]]


In [20]:
print(get_correlation([x6, x7]))

[[ 1.       -0.854559]
 [-0.854559  1.      ]]


### Calculate uncertainty contribution of base inputs

These are the sensitivity coefficients multiplied with the standard uncertainties of the base inputs x1 and x2.

In [21]:
print(get_jacobi(x7))

[[0.05109668 0.08131   ]]


### Calculate sensitivities w.r.t. intermediate results

x3 and x4 are intermediate results.

In [22]:
j1 = get_jacobi2(x7, [x3, x4])
print(j1)

[[0.30213326 0.10441677]]


NOTE: The calculation of sensitivities w.r.t. intermediate results
requires special care and can't be done blindly. There are two
requirements:
1. The vector g = [x3, x4] must be complete in the sense that x7 can be
calculated as a composite function x7 = f(g(x1,x2))
e.g. get_jacobi2(x7, x3) would return a wrong result
2. The elements of g must be linear independent
e.g. get_jacobi2(x7, [x3, x4]) with x4 = 2. * x3 would crash.
Generally it is not difficult to satisfy these requirement but it is 
the responsibility of the user to do so. You will not be warned and 
the returned result might be wrong, but it can be
checked by summing up uncertainty contributions and comparing with
combined uncertainty (see next cell).

### Calculate uncertainty contribution of intermediate result

In [23]:
u1 = get_unc_component(x7, [x3, x4])
print(u1)

[[0.06755905 0.02953352]]


this is the same as

In [24]:
print(j1 * get_stdunc([x3, x4]))

[[0.06755905 0.02953352]]


NOTE: the squared sum of these uncertainties is not equal to the 
combined uncertainty of x7, because x3 and x4 are correlated.

In [25]:
print(np.sqrt(np.sum(u1**2)))

0.07373231372057797


but by taking correlation into account it is the same as u(x7)

In [26]:
print(np.sqrt(np.dot(np.dot(j1, get_covariance([x3, x4])), j1.T)))

[[0.09603222]]


### Store an uncertainty object

stores uncertain objects in xml format maintaining all information

In [27]:
ustorage.save_xml_file([x6, x7], 'test.xml')

### Reload a stored uncertainty object

all information is recovered.

In [28]:
a1 = ustorage.load_xml_file('test.xml')

In [29]:
x6n = a1[0]
print(x6n)

1.5 ± 0.1118033988749895


In [30]:
x7n = a1[1]
print(x7n)

2.2304665972619078 ± 0.09603222054858558


correlation between x6n and x7n is the same as between x7 and x6

In [31]:
print(get_correlation([x6n, x7n]))

[[ 1.       -0.854559]
 [-0.854559  1.      ]]


also correlations with respect to any other quantities are the same

In [32]:
print(get_correlation([x6n, x7n, x6, x7, x3]))

[[ 1.        -0.854559   1.        -0.854559  -0.8      ]
 [-0.854559   1.        -0.854559   1.         0.9952598]
 [ 1.        -0.854559   1.        -0.854559  -0.8      ]
 [-0.854559   1.        -0.854559   1.         0.9952598]
 [-0.8        0.9952598 -0.8        0.9952598  1.       ]]


NOTE: x6n and x7n are in every aspect identical to x6 and x7. 
Recovery of full information works even if you shut down Python
and then restart, as long as the uncertainty objects were stored before 
shutdown and then reloaded after restart again.

All the necessary information is stored in the files.

### How to bridge non-analytical parts in the measurement equation

If the measurement equation consists of a non-analytical 
part, as e.g. a numerical method like nonlinear least squares, which 
returns an output p with value p0 and if the sensitivities j1 j2 of 
p with respect to the inputs x1 and x2 can be determined somehow, 
it is possible to define p as an uncertainty number with
p = ufloatsystem(p0, [x1, x2], [j1, j2]).

In [33]:
p0 = 5.0
j = [2.1, 4.5]
p = ufloatsystem(p0, [x1, x2], j)
print(p)

5.0 ± 0.9241753080449618


In [34]:
print(get_jacobi2(p, [x1, x2]))

[[2.1 4.5]]


## Higher order uncertainty propagation

Use the higer order uncertainty propagation amd specify the maximum order of the Taylor expansion.

In [35]:
use_distprop(maxlevel=3)

NOTE: There is no upper limit for maxlevel but be aware that memory 
consumption and computational burden increase exponentially with the
value of maxlevel.

maxlevel=3 is a good value which already provides reasonable
results for many cases at which linear uncertainty propagation fails.

### Creation of higher order uncertainty objects

the same as for linear uncertainty objects.

In the future it will be possible to specify distributions other than
Gaussians.

In [36]:
y1 = ufloat(0.0, 0.1)
print(y1)

0.0 ± 0.1


### Calculate with higher order uncertainty objects

Linear uncertainty propagation would return an uncertainty of 0.

In [37]:
y2 = y1**2
print(y2)

0.010000000000000002 ± 0.014142135623730954


In [38]:
y3 = umath.cos(y1)
print(y3)

0.995 ± 0.007071067811865477


### Expectation and function value are not the same

In [39]:
print(get_value(y2))

0.010000000000000002


In [40]:
print(get_fcn_value(y2))

0.0


In [41]:
print(get_value(y3))

0.995


In [42]:
print(get_fcn_value(y3))

1.0


### Uncertainties of higher order uncertainty objects

In [43]:
print(get_stdunc(y2))

0.014142135623730954


In [44]:
print(get_stdunc(y3))

0.007071067811865477


In [45]:
print(get_coverage_interval(y2, 0.95))

[[-0.01771808  0.03771808]]


In [46]:
print(get_coverage_interval(y3, 0.95))

[[0.98114096 1.00885904]]


NOTE: This is based on the assumption that the result distribution is 
Gaussian. It is possible to obtain central moments of the result 
distribution using get_moment(). This information is not yet used to 
calculate more realistic coverage intervals.